In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import json
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import plotly.io as pio
import plotly.express as px
print(pio.renderers)
pio.renderers.default = "notebook"

In [ ]:
cols = ["MachineName","SourceName","EventTime", "Message","Quality","Condition","Mask","NewState","Status"]
df = pd.read_excel("/home/waris/Github/tupras-analysis/new-alarm/proses_alarm.xls",skipinitialspace=True, usecols=cols)
df.tail(10)
# df.tail(10)


In [ ]:
# initial processing
print(type(df["EventTime"][i]))
for i in range(0, len(df["EventTime"])):
    d = df["EventTime"][i]
    d = d.replace(".000000000","")
    d = d.replace("/","-")
    df["EventTime"][i] = parse(d)
#     s = df["Message"][i]
#     df["Message"][i] = " ".join(s.split())

print(type(df["EventTime"][i]))

df.info()

In [ ]:
df.describe()

# client = MongoClient()
# db = client["tupras-db"]
# collection_raw_alarm = db["alarm"]
# data = json.loads(df.to_json(orient='records'))
# collection_raw_alarm.remove()
# print(data)
# collection_raw_alarm.insert(data)

# Preprocessing of Alarm data

There are different types of alarm data but Excel file only contains "Process Alarm Data".

**Excel Columns Description**
* MachineName: It represent the unit (e.g.,Plant-47)
* SourceName: The names of the devices operating in a unit (e.g., controller, transmitter, and calculator block names)
    1. Five types of controllers: TIC, PIC, FIC, LIC, and AIC.
    2. Three types of transmitters: TI, FI, LI
    3. Calculator block names ?

* EventTime: The date when an alarm is appeared. It also represent when it recovered right?

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state. 

**Questions:**
1. Before alarm data analysis, if ALM, NR and RECOVER messages are seen, these rows must be deleted. Why?
2. Alarm-off 
3. shelving of alarms
4. Can alarm cannot trigger twice? 
5. 

In [ ]:
sources = {}
source_conditions = {}
conditions = {} # Vel, IOP ALM, NR etc
key = ""
temp_dict = None
d = None
conditions["Recover"] = 0
for i in range(0, len(df["EventTime"])):
    key = df["SourceName"][i]
    temp_dict = {"Quality":df["Quality"][i], "Mask":df["Mask"][i], "NewState":df["NewState"][i], "Status": df["Status"][i] ,"SourceName": key,"EventTime": df["EventTime"][i], "Message":df["Message"][i],"Condition": df["Condition"][i]}
    
    if sources.get(key) != None:
        sources[key].append(temp_dict)
        
        #source conditions
        d =  source_conditions[key]
        if d.get(temp_dict["Condition"]) == None:
            d[temp_dict["Condition"]] = 0    
        d[temp_dict["Condition"]]  += 1
        d["Total"] += 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        # conditions           
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] += 1
        if temp_dict["Message"].strip().find("NR") != -1:
#             print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1
    else:
        sources[key] = []
        source_conditions[key] = {}
        sources[key].append(temp_dict)
        d = source_conditions[key]
        d[temp_dict["Condition"]] = 1
        d["Total"] = 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] +=1
        if temp_dict["Message"].strip().find("NR") != -1:
#             print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1


# sorting the source conditions dictionary based on the number of alarms related to each sensor
source_conditions = {k:v for k,v in sorted(source_conditions.items(), key=lambda item: item[1]["Total"],reverse=True)}
conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}


### Total number of disctinct SourceNames in the excel sheet.

In [ ]:

print("Total Number of Sourcenames: {s}\n\nSources List \n\n {l}".format(s=len(sources.keys()), l = sources.keys()))


### How many times each SourceName has been triggered (activation + Deactivation)?

In [ ]:

total  = 0
i =0
for key in source_conditions.keys():
    s1 = "Source: " + key + ", Total: {} | ".format(source_conditions[key]["Total"])
    s2 = ""
    total += source_conditions[key]["Total"]
    for k in source_conditions[key].keys():
        if k == "Total" or k == "Recover":
            continue
        s2 = s2 + k +" "+ str(source_conditions[key][k]) + ", "    
    s1 = s1 + s2
    print("{}. ".format(i),s1)
    i +=1
    if i == 10:
        break
print("\n\n Total Number of Entries in the excel:", total)
# assert(total== len(df["SourceName"]))

### Number of times each condition appear


In [ ]:
for k in conditions.keys():
    if k == "Recover":
        continue
    print(k,conditions[k])

print("Recover", conditions["Recover"]) ### including "NR" message

### Displaying Messages of top SourceName alarms (oreder by EventTime)
Which message is related to which message (i.e., activation and deactivation) ?


In [ ]:
# conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}

i = 0
for sname in source_conditions.keys():
    
    print("=======Source {}: {}========".format(i,sname))
    single_source_alarms = sources[sname]
    single_source_alarms = [v for v in sorted(single_source_alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    
    j = 0
    for alarm in single_source_alarms:
        print("{},---,{},--- Message: {}".format(alarm["EventTime"],alarm["Condition"], alarm["Message"]))
        j += 1
        if j == 30:
            break
    i += 1
    if i == 10:
        break


### **Find the duration of activation of each Alarm i.e., duration =  DeactivationTime - ActivationTime**

Compute mean, median, mode, on the duration...

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state.


In [ ]:
from datetime import timedelta
import statistics

def getDurationAndStats(alarms):
    """Returns list of dict and stats dictionary"""
#     queue = []
    conditions_queues = {}
    stats = {} 
    start_end = []
    d = None
    alarms = [v for v in sorted(alarms, key=lambda arg: arg["EventTime"], reverse = False)]
    for alarm in alarms:
#         print(alarm["Message"])
        if conditions_queues.get(alarm["Condition"]) == None:
            conditions_queues[alarm["Condition"]] = []

        if alarm["Message"].find("Recover") == -1 and alarm["Message"].find("NR") == -1:
            conditions_queues[alarm["Condition"]].append(alarm) # enqueue the activation
        else:
            if len(conditions_queues[alarm["Condition"]])== 0:
#                 print("(", alarm["SourceName"],alarm["Message"], ")")
                continue
            d = conditions_queues[alarm["Condition"]].pop(0) # dequeue
            d = {k:v for k,v in d.items()}
            d["StartTime"] = d["EventTime"]
            d["EndTime"] = alarm["EventTime"]
            d["EndMessage"] = alarm["Message"]
            del d["EventTime"]
            
            start_end.append(d)
#     print("Remaining Elements in Queue", len(queue))
    temps = []
    for alarm in start_end:
        temps.append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))

    stats["timedeltas"] = [v for v in sorted(temps)]
    stats["mean"] =  round(statistics.mean(stats["timedeltas"]),3) 
    stats["median"] = statistics.median(stats["timedeltas"])
    stats["mode"] = statistics.mode(stats["timedeltas"])
    stats["min"] = stats["timedeltas"][0]
    stats["max"] = stats["timedeltas"][-1]
    return (start_end, stats)

In [ ]:
i = 0
for key in source_conditions.keys():
    d = source_conditions[key]  
    print("{}. Source: {}, # of activations= {}, # of deactivations = {}, Diff = {}".format(i,key, d["Total"]-d["Recover"], d["Recover"], (d["Total"]-d["Recover"])-d["Recover"]))
    i += 1
    if i > 4:
        break

print ("==================================================================================")
i = 1
for sname in source_conditions.keys():
#     sname = "48TIC2026"
    if i > 10: 
        break
      
    alarms = sources[sname]
    _, stats = getDurationAndStats(alarms)
    print("{}. source:{}, Total alarms:{} | min = {}, max = {}, median = {}, mode = {}, mean = {}".format(i,sname,len(stats["timedeltas"]) ,stats["min"], stats["max"], stats["median"], stats["mode"], stats["mean"]))
    i += 1
print("Note: All the stats are in secods")

# Exporintg results to csv


In [ ]:
csv = {"SourceName":[],"StartTime":[], "EndTime": [] , "Message" : [], "RecoveryMessage":[] ,"Quality": [],"Condition":[],"Mask":[],"NewState": [],"Status": [],"TimeDelta": []} 

i = 1
for sname in source_conditions.keys():
#     sname = "48TIC2026"
    if i > 10: 
        break
    i += 1    
#     sname = "48TIC2026"

    alarms = sources[sname]
    alarms,stats = getDurationAndStats(alarms)

    for alarm in alarms:
        csv["SourceName"].append(alarm["SourceName"]) 
        csv["StartTime"].append(alarm["StartTime"]) 
        csv["EndTime"].append(alarm["EndTime"])
        csv["TimeDelta"].append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))
        csv["Message"].append(alarm["Message"])
        csv["RecoveryMessage"].append(alarm["EndMessage"])
        csv["Quality"].append(alarm["Quality"])
        csv["Condition"].append(alarm["Condition"])
        csv["Mask"].append(alarm["Mask"])
        csv["NewState"].append(alarm["NewState"])
        csv["Status"].append(alarm["Status"])


print("Total Entries :", len(csv["Status"]))
df = pd.DataFrame(csv)
df.to_csv("stats.csv", index = False)


In [ ]:

fig = px.box(csv, y="TimeDelta", color="SourceName",notched=True)
fig.show()

fig = px.violin(csv, y="TimeDelta", color="SourceName", box=True, points="all")
fig.show()

fig = px.scatter(csv, x="StartTime", y="TimeDelta",facet_col="SourceName", render_mode="webgl")
fig.show()

fig = px.density_heatmap(df, x="SourceName",y="TimeDelta", template="seaborn")
fig.show()

fig = px.scatter_matrix(csv, dimensions=["SourceName", "TimeDelta", "Condition"])
fig.show()

fig = px.parallel_coordinates(csv,  labels={"SourceName": "Source",
                  "StartTime": "Start","TimeDelta": "Delta", "Condition": "Condition",},
                    color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=2)
fig.show()
